# Fine Tune GPT-4o-mini with Custom Data.  

You will need a OpenAI API Key to move forward with this tutorial.

In [ ]:
!pip install --upgrade openai jsonlines

In [ ]:
import os
import time
from pprint import pprint
# OpenAI API Key
api_key =  os.environ["OPENAI_API_KEY"] = "<API KEY HERE>"

In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()

In [ ]:
%%time
items =[
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Listed media and gaming firm Nazara Technologies said on Friday that it acquired the remaining 48.42% stake it did not hold in Paper Boat Apps for Rs 300 crore, making it a wholly owned subsidiary."}, {"role": "assistant", "content": "{\"acquirer\": \" Nazara Technologies\", \"deal_size\": \"Rs 300 crore\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Venturi Partners (Venturi), a prominent growth equity firm specialising in consumer investments across India and Southeast Asia, announced the purchase of a $27 million stake in K12 Techno Services"}, {"role": "assistant", "content": "{\"acquirer\": Venturi Partners (Venturi)\", \"deal_size\": \"$27 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions."}, {"role": "assistant", "content": "{\"acquirer\": \"TechGiant Inc.\", \"deal_size\": \"$15 billion\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Future Ventures has completed its $3.5 million purchase of GreenTech Innovations."}, {"role": "assistant", "content": "{\"acquirer\": \"Future Ventures\", \"deal_size\": \"$3.5 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}, {"role": "assistant", "content": "{\"acquirer\": \"Leading Edge Capital\", \"deal_size\": \"$250 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Listed media and gaming firm Nazara Technologies said on Friday that it acquired the remaining 48.42% stake it did not hold in Paper Boat Apps for Rs 300 crore, making it a wholly owned subsidiary."}, {"role": "assistant", "content": "{\"acquirer\": \" Nazara Technologies\", \"deal_size\": \"Rs 300 crore\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Venturi Partners (Venturi), a prominent growth equity firm specialising in consumer investments across India and Southeast Asia, announced the purchase of a $27 million stake in K12 Techno Services"}, {"role": "assistant", "content": "{\"acquirer\": Venturi Partners (Venturi)\", \"deal_size\": \"$27 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions."}, {"role": "assistant", "content": "{\"acquirer\": \"TechGiant Inc.\", \"deal_size\": \"$15 billion\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Future Ventures has completed its $3.5 million purchase of GreenTech Innovations."}, {"role": "assistant", "content": "{\"acquirer\": \"Future Ventures\", \"deal_size\": \"$3.5 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}, {"role": "assistant", "content": "{\"acquirer\": \"Leading Edge Capital\", \"deal_size\": \"$250 million\" }"}]},

]

with jsonlines.open('mna.jsonl', 'w') as writer:
    writer.write_all(items)

In [ ]:
with jsonlines.open("mna.jsonl") as file:
    data = list(file.iter())

pprint(data)

In [ ]:
result_file = client.files.create(
  file=open("mna.jsonl", "rb"),
  purpose="fine-tune"
)

pprint(result_file)

In [ ]:
param_training_file_name = result_file.id
pprint(param_training_file_name)

In [ ]:
result_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18"
)

pprint(result_job)

In [ ]:
param_file_tune_job_id = result_job.id
pprint(param_file_tune_job_id)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status

In [ ]:
# Retrieve the state of a fine-tune
from time import sleep
while client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status != 'succeeded':
  sleep(10)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status

In [ ]:
param_file_tune_model = client.fine_tuning.jobs.retrieve(param_file_tune_job_id).fine_tuned_model
pprint(param_file_tune_model)

In [ ]:
import json

completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions!"}
  ]
)
pprint(json.loads(completion.choices[0].message.content))

In [ ]:
completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}
  ]
)
pprint(json.loads(completion.choices[0].message.content))

In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
result_delete = client.models.delete(param_file_tune_model)
pprint(result_delete)